In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # connect to default database
    #conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=passer")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify databasewith utf8 encoding
    cur.execute("DROP DATABASE creditbureau")
    cur.execute("CREATE DATABASE creditbureau")
    
    # close connection to default database
    conn.close()
    
    # connect connection tow defaul database
    conn = psycopg2.connect("host=127.0.0.1 dbname=creditbureau user=postgres password=passer")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

Une agence d'évaluation (Credit Bureau) du crédit est une agence de collecte de données qui recueille des informations sur les comptes de divers créanciers et fournit ces informations à une agence de renseignements, d'evaluation de credit etc...

In [5]:
credit_bureau = pd.read_csv("bureau.csv")

In [6]:
credit_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [7]:
credit_bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [8]:
bureau_balance = pd.read_csv("bureau_balance.csv")

In [9]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [11]:
cur, conn = create_database()

In [12]:
credit_bureau_table_create = ("""CREATE TABLE IF NOT EXISTS bureau(
SK_ID_CURR VARCHAR,
SK_ID_BUREAU VARCHAR PRIMARY KEY,
CREDIT_ACTIVE VARCHAR,
CREDIT_CURRENCY VARCHAR,
DAYS_CREDIT numeric,
CREDIT_DAY_OVERDUE numeric,
DAYS_CREDIT_ENDDATE numeric,
DAYS_ENDDATE_FACT numeric,
AMT_CREDIT_MAX_OVERDUE numeric,
CNT_CREDIT_PROLONG numeric,
AMT_CREDIT_SUM numeric,
AMT_CREDIT_SUM_DEBT numeric,
AMT_CREDIT_SUM_LIMIT numeric,
AMT_CREDIT_SUM_OVERDUE numeric,
CREDIT_TYPE VARCHAR,
DAYS_CREDIT_UPDATE numeric,
AMT_ANNUITY numeric
)""")

In [13]:
cur.execute(credit_bureau_table_create)
conn.commit()

In [14]:
bureau_balance_table_create = ("""CREATE TABLE IF NOT EXISTS bureau_balance(
SK_ID_BUREAU VARCHAR,
MONTHS_BALANCE numeric,
STATUS VARCHAR
)""")

In [15]:
cur.execute(bureau_balance_table_create)
conn.commit()

In [16]:
credit_bureau_table_insert = ("""INSERT INTO bureau(
SK_ID_CURR,
SK_ID_BUREAU,
CREDIT_ACTIVE,
CREDIT_CURRENCY,
DAYS_CREDIT,
CREDIT_DAY_OVERDUE,
DAYS_CREDIT_ENDDATE,
DAYS_ENDDATE_FACT,
AMT_CREDIT_MAX_OVERDUE,
CNT_CREDIT_PROLONG,
AMT_CREDIT_SUM,
AMT_CREDIT_SUM_DEBT,
AMT_CREDIT_SUM_LIMIT,
AMT_CREDIT_SUM_OVERDUE,
CREDIT_TYPE,
DAYS_CREDIT_UPDATE,
AMT_ANNUITY)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)
""")

In [17]:
for i, row in credit_bureau.iterrows():
    cur.execute(credit_bureau_table_insert, list(row))

In [18]:
bureau_balance_table_insert = ("""INSERT INTO bureau_balance(
SK_ID_BUREAU,
MONTHS_BALANCE,
STATUS)
VALUES (%s, %s, %s)
""")

In [22]:
for i, row in bureau_balance.iterrows():
    cur.execute(bureau_balance_table_insert, list(row))

In [21]:
conn.commit()